In [ ]:
ADDR_I_CS_DATA      = 0x10
BITS_I_CS_DATA      = 1
ADDR_I_WE_DATA      = 0x18
BITS_I_WE_DATA      = 1
ADDR_I_ADDRESS_DATA = 0x20
BITS_I_ADDRESS_DATA = 8
ADDR_I_WRITE_DATA   = 0x28
BITS_I_WRITE_DATA   = 32
ADDR_O_READ_DATA    = 0x30
BITS_O_READ_DATA    = 32
ADDR_O_READ_CTRL    = 0x34
ADDR_O_ERROR_DATA   = 0x38
BITS_O_ERROR_DATA   = 1
ADDR_O_ERROR_CTRL   = 0x3c

from pynq import Overlay

overlay = Overlay('/home/xilinx/sha1_overlay/sha1_overlay.bit')

In [ ]:
overlay?

In [ ]:
sha1_ip = overlay.sha1_control_0
sha1_ip?

In [ ]:
def check_name():
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 0)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name0 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name0)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 1)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    name1 = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(name1)[2:]).decode())
    
    sha1_ip.write(ADDR_I_ADDRESS_DATA, 2)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    version = sha1_ip.read(ADDR_O_READ_DATA)
    print(bytearray.fromhex(hex(version)[2:]).decode())

In [ ]:
def read_word(address):
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 0)
    while(sha1_ip.read(ADDR_O_READ_DATA) != 0):
        word = sha1_ip.read(ADDR_O_READ_DATA)
        break
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    return word

In [ ]:
def write_word(address, word):
    # add error handling word (32 bits), address (8 bits)
    sha1_ip.write(ADDR_I_ADDRESS_DATA, address)
    sha1_ip.write(ADDR_I_WRITE_DATA, word)
    sha1_ip.write(ADDR_I_CS_DATA, 1)
    sha1_ip.write(ADDR_I_WE_DATA, 1)
    sha1_ip.write(ADDR_I_CS_DATA, 0)
    sha1_ip.write(ADDR_I_WE_DATA, 0)

In [ ]:
def write_block(block):
    # block is 512 bits
    mask = 0xffffffff000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
    write_word(0x10, (block & mask) >> 480);
    write_word(0x11, (block & (mask >> 32)) >> 448);
    write_word(0x12, (block & (mask >> 64)) >> 416);
    write_word(0x13, (block & (mask >> 96)) >> 384);
    write_word(0x14, (block & (mask >> 128)) >> 352);
    write_word(0x15, (block & (mask >> 160)) >> 320);
    write_word(0x16, (block & (mask >> 192)) >> 288);
    write_word(0x17, (block & (mask >> 224)) >> 256);
    write_word(0x18, (block & (mask >> 256)) >> 224);
    write_word(0x19, (block & (mask >> 288)) >> 192);
    write_word(0x1a, (block & (mask >> 320)) >> 160);
    write_word(0x1b, (block & (mask >> 352)) >> 128);
    write_word(0x1c, (block & (mask >> 384)) >> 96);
    write_word(0x1d, (block & (mask >> 416)) >> 64);
    write_word(0x1e, (block & (mask >> 448)) >> 32);
    write_word(0x1f, block & (mask >> 480));

In [ ]:
def read_digest():
    digest_data_mask = 0xffffffff000000000000000000000000ffffffff
    mask = 0xffffffff00000000000000000000000000000000
    digest_data = (read_word(0x20) & digest_data_mask) << 128
    digest_data = digest_data | ((read_word(0x21) & digest_data_mask) << 96)
    digest_data = digest_data | ((read_word(0x22) & digest_data_mask) << 64)
    digest_data = digest_data | ((read_word(0x23) & digest_data_mask) << 32)
    digest_data = digest_data | read_word(0x24)
    return digest_data

In [ ]:
check_name()

In [ ]:
test = read_word(0)
print(test)
print(bytearray.fromhex(hex(test)[2:]).decode())

In [ ]:
write_word(0x10, 5)
test = read_word(0x10)
print(test)

In [ ]:
def test_sha1():
    # single block msg = "abc"
    # expected output = 968236873715988614170569073515315707566766479517
    test_block = 0x61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
    write_block(test_block)
    write_word(0x08, 1)
    digest = read_digest()
    print(digest)

In [ ]:
test_sha1()